# A575 HW #4
### Armaan Goyal

In [ ]:
#basic packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import scipy

#specific tasks and routines
from matplotlib.offsetbox import AnchoredText
from scipy.integrate import quad
from scipy.optimize import least_squares
from scipy.optimize import curve_fit

#plotting and LaTeX
from astropy.visualization import astropy_mpl_style
plt.style.use("ggplot")
mpl.rcParams["figure.dpi"]=100
%config InlineBackend.figure_format = "svg"
plt.rcParams["text.usetex"] =True

In [ ]:
# Magnitude conversions
def M_to_m(M, d):
    return M + 5*np.log10(d) - 5

def m_to_M(m, d):
    return m + 5 - 5*np.log10(d)

# Generate Mock Galaxies
N = 100000
max_d = 1e8

M = np.random.normal(-19, 2, N)

# distances
coords = np.random.uniform(0, max_d, (3, 3*N))
d = np.sqrt(np.sum(coords*coords, axis = 0))
mask = (d < 1e8)
d = d[mask][:N]
m = M_to_m(M, d)

# histogram limits & bin size
Mmin = -11
Mmax = -27
NM = np.abs(Mmin-Mmax)*4

counts_full = np.histogram(M, NM, (Mmax, Mmin))[0]
bins = np.histogram(M, NM, (Mmax, Mmin))[1]

# local volume
local_mask = (d < 3e7)
M_local = M[local_mask]
counts_local = np.histogram(M_local, NM, (Mmax, Mmin))[0]

In [ ]:
# apply limiting magnitude, caluculate mean dist & abs mag, completeness limit
def mag_limited(M, d, m, mag_cut, dist_cut):

    mask = (m <= mag_cut)
    M_cut, d_cut, m_cut = M[mask], d[mask], m[mask]
    print("For cutoff of m = %d mag"%(mag_cut))
    print("Mean Absolute Magniutde: %.2f mag"%(np.mean(M_cut)))
    print("Mean Distance: %.2f Mpc"%(np.mean(d_cut)/(1e6)))

    local_mask = (d_cut < dist_cut)
    M_cut_local = M_cut[local_mask]
    counts_cut_local = np.histogram(M_cut_local, NM, (Mmax, Mmin))[0]
    diff = counts_local - counts_cut_local
    bin_mask = (diff != 0)
    which_bin = bins[:-1][bin_mask][0]
    print("Sample complete to M = %.2f for local volume"%(which_bin))
    
    return M_cut, d_cut, m_cut

In [ ]:
# m < 16
M_16, d_16, m_16 = mag_limited(M, d, m, 16, 3e7)

In [ ]:
# m < 14
M_14, d_14, m_14 = mag_limited(M, d, m, 14, 3e7)

In [ ]:
# m < 12
M_12, d_12, m_12 = mag_limited(M, d, m, 12, 3e7)

In [ ]:
# plot histograms and M vs. d
fig, ax = plt.subplots(2, 3, figsize=(10,8), sharey="row")
props = dict(alpha=0)
        
ax[0][0].text(0.5, 0.15, "$\overline{d}$ = %.2f Mpc"%(np.mean(d_16)/(1e6)), transform=ax[0][0].transAxes, fontsize=12, verticalalignment='top', bbox=props)
ax[0][0].scatter(d_16/1e6, M_16)
ax[0][0].invert_yaxis()
ax[0][0].set_ylabel("$M$ [mag]")
ax[0][0].title.set_text("$m \leq 16$ mag")

ax[0][1].text(0.5, 0.15, "$\overline{d}$ = %.2f Mpc"%(np.mean(d_14)/(1e6)), transform=ax[0][1].transAxes, fontsize=12, verticalalignment='top', bbox=props)
ax[0][1].scatter(d_14/1e6, M_14)
ax[0][1].invert_yaxis()
ax[0][1].set_xlabel("$d$ [Mpc]")
ax[0][1].title.set_text("$m \leq 14$ mag")

ax[0][2].text(0.5, 0.15, "$\overline{d}$ = %.2f Mpc"%(np.mean(d_12)/(1e6)), transform=ax[0][2].transAxes, fontsize=12, verticalalignment='top', bbox=props)
ax[0][2].scatter(d_12/1e6, M_12)
ax[0][2].invert_yaxis()
ax[0][2].title.set_text("$m \leq 12$ mag")
#fig.show()
        
ax[1][0].text(0.05, 0.8, "$\overline{M}$ = %.2f mag"%(np.mean(M_16)), transform=ax[1][0].transAxes, fontsize=10, verticalalignment='top', bbox=props)
ax[1][0].hist(M_16, bins)
ax[1][0].invert_xaxis()
ax[1][0].set_ylabel("Counts")
ax[1][0].axvline(np.mean(M_16), 0, 1000, ls = "--", color = "black")

ax[1][1].text(0.05, 0.65, "$\overline{M}$ = %.2f mag"%(np.mean(M_14)), transform=ax[1][1].transAxes, fontsize=11, verticalalignment='top', bbox=props)
ax[1][1].hist(M_14, bins)
ax[1][1].invert_xaxis()
ax[1][1].set_xlabel("$M$ [mag]")
ax[1][1].axvline(np.mean(M_14), 0, 1000, ls = "--", color = "black")

ax[1][2].text(0.05, 0.65, "$\overline{M}$ = %.2f mag"%(np.mean(M_12)), transform=ax[1][2].transAxes, fontsize=11, verticalalignment='top', bbox=props)
ax[1][2].hist(M_12, bins)
ax[1][2].invert_xaxis()
ax[1][2].axvline(np.mean(M_12), 0, 1000, ls = "--", color = "black")
fig.show()
fig.savefig("mag_histograms", dpi = 400)

In [ ]:
# define linear relationships

def P_to_M(P, C1, C2):
    return C1*P + C2

def M_to_P(M, C1, C2):
    return (M-C2)/C1

# generate "true" and noisy P-values
P_true = M_to_P(M, -7.27, -20)
P = np.random.normal(P_true, .3/7.27)

In [ ]:
# plot M vs. P
fig = plt.figure()
plt.scatter(P, M)
plt.title("$M$ vs. $P$ for Total Sample")
plt.ylabel("$M$ [mag]")
plt.xlabel("$P$ [arb. units]")
plt.gca().invert_yaxis()
plt.show()
fig.savefig("M_vs_P", dpi = 400)

In [ ]:
# Plot residuals for M (total sample)

M_P = P_to_M([-7.27, -20], P)
diff = M - M_P

fig, ax = plt.subplots(1, 3, figsize=(10,4), sharey=True)
fig.subplots_adjust(hspace = .5, wspace=.1)
fig.suptitle("$(M-M_{P})$ Behavior for Total Sample", fontsize = 16)
props = dict(alpha=0)
    
ax[0].scatter(M, diff)
ax[0].set_ylabel("$(M-M_{P})$ [mag]")
ax[0].set_xlabel("$M$ [mag]")
ax[0].invert_xaxis()
ax[0].invert_yaxis()

ax[1].scatter(P, diff)
ax[1].set_xlabel("$P$ [arb. units]")
ax[1].invert_yaxis()

ax[2].scatter(d/1e6, diff)
ax[2].set_xlabel("$d$ [Mpc]")
ax[2].invert_yaxis()
fig.show()
fig.savefig("M_MP_total", dpi=400)

In [ ]:
# least-squares fitting in M and P

def fitting(P, M):
    popt_M, pcov_M = curve_fit(P_to_M, P, M, [-7.27, -20])
    perr_M = np.sqrt(np.diag(pcov_M))
    
    popt_P, pcov_P = curve_fit(M_to_P, M, P, [-7.27, -20])
    perr_P = np.sqrt(np.diag(pcov_P))
    
    print("Minimizing in P yields C1 = %.3f \u00b1 %.3f and C2 = %.3f \u00b1 %.3f"%(popt_P[0], perr_P[0], popt_P[1], perr_P[1]))
    print("Minimizing in M yields C1 = %.3f \u00b1 and %.3f C2 = %.3f \u00b1 %.3f "%(popt_M[0], perr_M[0], popt_M[1], perr_M[1]))
    
    return popt_M, popt_P, perr_M, perr_P

In [ ]:
# m < 16
P_16_true = M_to_P(M_16, -7.27, -20)
P_16 = np.random.normal(P_16_true, .3/7.27)

M_params_16, P_params_16, err_M_16, err_P_16 = fitting(P_16, M_16)

In [ ]:
# m < 14
P_14_true = M_to_P(M_14, -7.27, -20)
P_14 = np.random.normal(P_14_true, .3/7.27)

M_params_14, P_params_14, err_M_14, err_P_14 = fitting(P_14, M_14)

In [ ]:
# m < 12
P_12_true = M_to_P(M_12, -7.27, -20)
P_12 = np.random.normal(P_12_true, .3/7.27)

M_params_12, P_params_12, err_M_12, err_P_12 = fitting(P_12, M_12)

In [ ]:
# plot fits for mag-limited samples

fig, ax = plt.subplots(1, 3, figsize=(10,4), sharey=True)
fig.subplots_adjust(hspace = .5, wspace=.1)
props = dict(alpha=0)

ax[0].scatter(P_16, M_16)
ax[0].plot(P_16, P_to_M(P_16, M_params_16[0], M_params_16[1]), color = "blue", label = "Minimization in $M$")
ax[0].plot(M_to_P(M_16, P_params_16[0], P_params_16[1]), M_16, color = "lime", label = "Minimization in $P$")
ax[0].text(0.40, 0.2, "$x_{M}$ = (%.2f, %.2f)"%(M_params_16[0], M_params_16[1]), transform=ax[0].transAxes, fontsize=12, verticalalignment='top', bbox=props)
ax[0].text(0.40, 0.1, "$x_{P}$ = (%.2f, %.2f)"%(P_params_16[0], P_params_16[1]), transform=ax[0].transAxes, fontsize=12, verticalalignment='top', bbox=props)
ax[0].title.set_text("$m \leq 16$ mag")
ax[0].set_ylabel("$M$ [mag]")
ax[0].invert_yaxis()
ax[0].legend()

ax[1].scatter(P_14, M_14)
ax[1].plot(P_14, P_to_M(P_14, M_params_14[0], M_params_14[1]), color = "blue", label = "Minimization in $M$")
ax[1].plot(M_to_P(M_14, P_params_14[0], P_params_14[1]), M_14, color = "lime", label = "Minimization in $P$")
ax[1].text(0.40, 0.2, "$x_{M}$ = (%.2f, %.2f)"%(M_params_14[0], M_params_14[1]), transform=ax[1].transAxes, fontsize=12, verticalalignment='top', bbox=props)
ax[1].text(0.40, 0.1, "$x_{P}$ = (%.2f, %.2f)"%(P_params_14[0], P_params_14[1]), transform=ax[1].transAxes, fontsize=12, verticalalignment='top', bbox=props)
ax[1].title.set_text("$m \leq 14$ mag")
ax[1].set_xlabel("$P$ [arb. units]")
ax[1].invert_yaxis() 
    
ax[2].scatter(P_12, M_12)
ax[2].plot(P_12, P_to_M(P_12, M_params_12[0], M_params_12[1]), color = "blue", label = "Minimization in $M$")
ax[2].plot(M_to_P(M_12, P_params_12[0], P_params_12[1]), M_12, color = "lime", label = "Minimization in $P$")
ax[2].text(0.40, 0.2, "$x_{M}$ = (%.2f, %.2f)"%(M_params_12[0], M_params_12[1]), transform=ax[2].transAxes, fontsize=12, verticalalignment='top', bbox=props)
ax[2].text(0.40, 0.1, "$x_{P}$ = (%.2f, %.2f)"%(P_params_12[0], P_params_12[1]), transform=ax[2].transAxes, fontsize=12, verticalalignment='top', bbox=props)
ax[2].title.set_text("$m \leq 12$ mag")
ax[2].invert_yaxis() 

fig.show()
fig.savefig("M_P_fitting", dpi=400)

In [ ]:
# plot residuals for m < 12 sample

M_P_12_good = P_to_M(P_params_12, P_12)
M_P_12_bad = P_to_M(M_params_12, P_12)
diff_good = M_12 - M_P_12_good
diff_bad = M_12 - M_P_12_bad

fig, ax = plt.subplots(1, 3, figsize=(10,4), sharey=True)
fig.subplots_adjust(hspace = .5, wspace=.1)
fig.suptitle("$(M-M_{P})$ Behavior for $m \leq 12$ mag", fontsize = 16)
props = dict(alpha=0)

ax[0].scatter(M_12, diff_good, label = "Minimization in P")
ax[0].scatter(M_12, diff_bad, label = "Minimization in M")
ax[0].set_ylabel("$(M-M_{P})$ [mag]")
ax[0].set_xlabel("$M$ [mag]")
ax[0].invert_xaxis()
ax[0].invert_yaxis()
ax[0].legend()

ax[1].scatter(P_12, diff_good, label = "Minimization in P")
ax[1].scatter(P_12, diff_bad, label = "Minimization in M")
ax[1].set_xlabel("$P$ [arb. units]")
ax[1].invert_yaxis()

ax[2].scatter(d_12/1e6, diff_good, label = "Minimization in P")
ax[2].scatter(d_12/1e6, diff_bad, label = "Minimization in M")
ax[2].set_xlabel("$d$ [Mpc]")
ax[2].invert_yaxis()
fig.show()
fig.savefig("good_bad_fitting", dpi=400)